Shiksha Saarthi: Advanced Multi-Agent Exam Preparation Tutor

**Project Overview**


Track: Agents for Good


Problem: Standard online learning is generic. Students preparing for high-stakes competitive exams (like GATE, NIMCET) need personalized, real-difficulty level coaching in their preferred language.




Solution: Shiksha Saarthi is a Multi-Agent system that acts as a specialized coach. It uses three distinct agents to provide adaptive, exam-specific lessons, detailed solutions, and continuous performance tracking.




In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install google-genai

  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1


In [ ]:


import os
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient

# Load the API Key from Kaggle Secrets
try:
    secrets = UserSecretsClient()
    os.environ["GEMINI_API_KEY"] = secrets.get_secret("GEMINI_API_KEY") 
    print("Gemini API Key successfully loaded.")
except Exception as e:
    print(f"Error loading API Key: {e}")
    print("Please set GEMINI_API_KEY in Kaggle Secrets.")
    
# Initialize the Gemini Client
client = genai.Client()

Gemini API Key successfully loaded.


Agent 2: Diagnostic Agent (Sessions & Memory)

This agent is the system's memory.


Function: It tracks the student's progress (score_total, question_count) and stores the complete Q&A history.

Architecture Role: It informs the Teacher Agent whether the student needs a harder topic or remedial help (real-world context). (Fulfills Mandatory Feature: Sessions & Memory)



In [10]:
# DiagnosticAgent Code
class DiagnosticAgent:
    """
    Diagnostic Agent tracks the student's progress, including questions, answers, and results. (Memory Feature)
    """
    def __init__(self):
        self.student_data = {} 

    def retrieve_state(self, student_id):
        return self.student_data.get(student_id, None)

    def update_progress(self, student_id, topic, is_correct, question_asked=None, user_answer=None):
        """ Updates progress based on the student's answer, saving the question and answer. """
        if student_id not in self.student_data:
            self.student_data[student_id] = {
                "progress": [],
                "current_topic": topic,
                "score_total": 0,
                "question_count": 0
            }
            
        data = self.student_data[student_id]
        
        # Detailed progress data is included here
        data["progress"].append({
            "topic": topic,
            "question": question_asked,
            "answer_given": user_answer,
            "correct": is_correct
        })
        
        data["question_count"] += 1
        if is_correct:
            data["score_total"] += 1
        
        data["current_topic"] = topic
        
    def analyze_and_suggest_next_step(self, student_id):
        """ Suggests the next step for the Teacher Agent. """
        data = self.student_data.get(student_id)
        
        if not data or data["question_count"] < 1:
            return {"next_topic": "Math: Basic Addition", "need_real_world_example": False}
        
        score_percent = (data["score_total"] / data["question_count"]) * 100
        current_topic_full = data["current_topic"]
        
        try:
            current_subject = current_topic_full.split(":")[0].strip()
        except IndexError:
             current_subject = "Math" 

        
        if score_percent < 60:
            # If the score is low, request help from Research Agent (stay on the same topic)
            return {
                "next_topic": current_topic_full,
                "confidence_score": f"{score_percent:.0f}%",
                "need_real_world_example": True, 
                "reason": "Low score, needs more engaging content."
            }
        else:
            # If the score is good, suggest moving to the next topic (based on subject)
            if "Science" in current_subject:
                next_topic = "Science: The Water Cycle"
            elif "History" in current_subject:
                 next_topic = "History: Feudalism"
            else: # Math
                next_topic = "Math: Basic Fractions"
            
            return {
                "next_topic": next_topic, 
                "confidence_score": f"{score_percent:.0f}%",
                "need_real_world_example": False,
                "reason": "Student shows good understanding, ready for next concept."
            }

diagnostic_agent = DiagnosticAgent()

Agent 3: Research Agent (Tools Integration)

This agent integrates external, real-time data.

Function: It uses the Google Search Tool to fetch real-world examples (Class Mode) or confirm the required PYQ difficulty/syllabus context for the specific exam (Competitive Mode).

Architecture Role: It provides contextual data to the Teacher Agent to make lessons more relevant. (Fulfills Mandatory Feature: Tools)


In [11]:
# Cell 3: ResearchAgent Code (CRITICAL FIX for Server Errors)
from google import genai
from google.genai import types
from google.genai.errors import APIError, ServerError # IMPORT CRITICAL FOR HANDLING ERRORS

class ResearchAgent:
    """
    Research Agent finds external information using the Google Search Tool (Tools Feature).
    """
    def __init__(self, client):
        self.client = client
        self.system_instruction = (\
            "You are the Research Agent. Your only job is to receive a search query, "\
            "use the google_search tool to find information, and summarize the results "\
            "in a simple, child-friendly format (primary education level). "\
            "Your output must be the summarized text only."\
        )

    def execute_search(self, search_query: str) -> str:
        """ Executes the query using the Google Search Tool, with strict error handling. """
        
        print(f"\n[Research Agent]: Attempting to search for '{search_query}'...")
        
        try:
            # Attempt to use the Gemini model with the Google Search Tool
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=[
                    self.system_instruction,
                    f"Please find a simple, real-world context for: {search_query}"
                ],
                config=types.GenerateContentConfig(
                    tools=[{"google_search": {}}] 
                )
            )
            
            if response.text:
                print("[Research Agent]: Summary generated successfully.")
                return response.text
            else:
                print("[Research Agent]: Could not generate a summary (no text returned).")
                return "Context search failed: Could not generate a summary."

        # CATCH ALL API AND SERVER ERRORS! (THIS IS THE FIX)
        except (APIError, ServerError, Exception) as e:
            # CRITICAL FALLBACK: If the API call fails, return a safe string immediately.
            print(f"[Research Agent CRITICAL ERROR]: Search failed. Returning default context. Error: {e}")
            return "Context search failed: Returning a generic internal example due to an external server error."

research_agent = ResearchAgent(client)

Agent 4: Teacher Agent (Multi-Agent Coordinator)

This is the core LLM-powered agent and the system coordinator.

Function: It orchestrates the flow: 1. Gets next step from Diagnostic Agent. 2. Requests context from Research Agent. 3. Generates the final lesson/question using advanced prompts for the chosen mode (Class/Competitive).

Architecture Role: It demonstrates the system's Sequential Agent structure. (Fulfills Mandatory Feature: Multi-agent system)


In [12]:
# TeacherAgent Code
class TeacherAgent:
    """
    Teacher Agent handles the main conversation and is the core of the Multi-Agent System.
    """
    def __init__(self, client, diagnostic_agent, research_agent):
        self.client = client
        self.diagnostic_agent = diagnostic_agent
        self.research_agent = research_agent
        self.student_id = "live_student" 
        self.current_topic = None 
        
        self.system_instruction = (
            "You are the Teacher Agent, 'Shiksha Saarthi.' You are teaching primary school concepts. "
            "Be patient, encouraging, and always follow the instructions given by the Diagnostic Agent for the next lesson."
        )

    def start_session(self, mode, language, exam_name=None): 
        """ Starts the session and coordinates the next lesson/question. """
        
        suggestion = self.diagnostic_agent.analyze_and_suggest_next_step(self.student_id)
        next_topic = suggestion["next_topic"]
        self.current_topic = next_topic 
        
        print(f"\n[DIAGNOSTICS]: Suggested next topic is: {next_topic}")
        
        extra_context = None
        
        # Research Agent usage for Competitive Mode
        if mode == "Competitive":
            search_query = f"Previous year question example for {next_topic} in {exam_name}"
            extra_context = self.research_agent.execute_search(search_query)
            print(f"[TEACHER]: Using competitive context from Research Agent.")
            
        elif suggestion["need_real_world_example"]:
            # Research Agent usage for improvement in Class Mode
            search_query = f"Simple real-world examples for {next_topic}"
            extra_context = self.research_agent.execute_search(search_query)
            
        # Pass language to generate_lesson
        return self.generate_lesson(next_topic, mode, language, exam_name, extra_context=extra_context)


    def generate_lesson(self, topic, mode, language, exam_name=None, extra_context=None):
        """ Generates the lesson/question using Gemini, adapting based on mode and language. """
        
        # Add language instruction to the top of the prompt
        language_instruction = f"All output MUST be in {language} language."
        
        if mode == "Competitive":
            # Detailed prompt for Competitive Mode
            prompt = (
                f"{language_instruction} Topic: {topic}. You are acting as an expert coach for the {exam_name} exam. "
                "The question must match the REAL difficulty level of {exam_name}."
                "First, provide a brief, exam-focused concept review (padhai). "
                "Then, ask ONE challenging question. "
                "If the student answers incorrectly, you MUST provide a step-by-step, exact solution and explanation (samjha ke) like an expert would."
            )
        else:
            # Prompt for Class Mode (Default)
            prompt = (
                f"{language_instruction} Topic: {topic}. You are acting as a school teacher. "
                "Create a simple lesson and ask one question that tests this basic concept."
            )

        if extra_context:
            prompt += f"\n\nUse this context (Research Agent found PYQ/Syllabus info): {extra_context}"
            
        
        lesson_response = self.client.models.generate_content(
            model='gemini-2.5-flash',
            contents=[self.system_instruction, prompt]
        )
        return lesson_response.text

    def evaluate_answer(self, user_answer, question_text, language): 
        """ Evaluates the student's answer. """
        
        topic = self.current_topic 
        
        evaluation_prompt = (
            f"All feedback and responses MUST be in {language} language. Topic: {topic}. Student's answer: '{user_answer}'. "
            "Did the student understand the core concept? Respond with only the word TRUE or FALSE (first line), and provide encouraging feedback (second line)."
        )
        
        evaluation_response = self.client.models.generate_content(
            model='gemini-2.5-flash',
            contents=[self.system_instruction, evaluation_prompt] 
        )
        
        is_correct = "TRUE" in evaluation_response.text.split('\n')[0].upper()
        
        # Send data to the Diagnostic Agent
        self.diagnostic_agent.update_progress(
            self.student_id, 
            topic, 
            is_correct, 
            question_asked=question_text,  
            user_answer=user_answer
        )
        
        return is_correct, evaluation_response.text

    def get_current_topic(self):
        return self.current_topic
        
    def get_diagnostic_report(self):
        return self.diagnostic_agent.retrieve_state(self.student_id)

teacher_agent = TeacherAgent(client, diagnostic_agent, research_agent)

System Architecture Flow


Your system operates sequentially:

1. User Input (Mode, Language, Topic).


2. Teacher Agent queries Diagnostic Agent for next step.


3. Teacher Agent queries Research Agent for real-time context/PYQ.


4. Teacher Agent uses all context to generate the lesson/question.


5. Teacher Agent evaluates the answer and updates Diagnostic Agent memory.

In [13]:
# --- Interactive Chat Loop (PURE INTERACTIVE MODE with Y/N Check) ---
import json
import sys

# Dictionary of subjects and their initial topics
DEFAULT_TOPICS = {
    "math": "Basic Addition",
    "science": "Basic Plants/Animals",
    "history": "Ancient Civilizations",
    "analytical ability": "Basic Reasoning",
    "computer awareness": "Basic Terminology",
    "english": "Basic Grammar"
}

# In pure interactive mode, we don't need mock input logic.
def mock_input(prompt):
    return "exit"

def interactive_chat_loop(is_notebook_save=False):
    # --- PURE INTERACTIVE SETUP ---
    # We ignore the save check and force input_func to be the real 'input'.
    input_func = input
    is_notebook_save = False 
    # --- END PURE INTERACTIVE SETUP ---
    
    print("-----------------------------------------------------")
    print("🤖 Shiksha Saarthi (Interactive Session) 🚀")
    print("Type 'exit' or 'quit' to end the session at any time.") 
    print("-----------------------------------------------------")


    # --- 0. Language Selection (OLD FLOW) ---
    session_language = input_func("In which language would you like to study (e.g., Hindi, English)? ").strip()
    if not session_language:
        session_language = "English"
        print(f"Default language: {session_language} selected.")

    # --- 1. Mode Selection (OLD FLOW) ---
    exam_name = None
    while True:
        mode_choice = input_func("Which mode would you like to use (Class/Competitive)? ").strip()
        if mode_choice.lower() == 'class':
            session_mode = 'Class'
            break
        elif mode_choice.lower() == 'competitive':
            session_mode = 'Competitive'
            exam_name = input_func("Which Competitive Exam are you preparing for (e.g., GATE, NIMCET, CTPG)? ").strip()
            if not exam_name:
                exam_name = "Generic Competitive Exam" 
            break
        else:
            print("❌ Please choose either Class or Competitive.")

    # --- 2. Subject Selection (OLD FLOW) ---
    subject_choice = input_func("Which subject would you like to study? ").strip()
    subject_key = subject_choice.lower()
    initial_subject = subject_key.capitalize()

    # --- 3. Topic Selection (OLD FLOW) ---
    default_topic = DEFAULT_TOPICS.get(subject_key, "Basic Concepts")
    topic_choice = input_func(f"Which topic in {initial_subject} would you like to study (Press Enter for default: {default_topic})? ").strip()

    if not topic_choice:
        initial_topic = f"{initial_subject}: {default_topic}"
    else:
        initial_topic = f"{initial_subject}: {topic_choice.strip()}"

    print(f"\nStarting {session_mode} session for: {initial_topic} in {session_language} (Exam: {exam_name})...\n")

    # Set the Diagnostic Agent's memory (initial dummy entry)
    teacher_agent.diagnostic_agent.update_progress(teacher_agent.student_id, initial_topic, False, question_asked="Session Start", user_answer="N/A")
    teacher_agent.current_topic = initial_topic 

    # 4. Get the first lesson
    previous_lesson = teacher_agent.start_session(session_mode, session_language, exam_name) 
    print("\n[TEACHER]: " + previous_lesson)

    # Conversation loop
    while True:
        try:
            # Determine user input (only real input is used here)
            user_input = input_func("\n[STUDENT]: ") # <--- सिंगल प्रॉम्प्ट जैसा आप चाहते थे
            
            # --- 1. EXIT LOGIC (Answer Box Exit) ---
            if user_input.lower() in ['exit', 'quit']:
                print("\nThank you! Your session has ended.")
                print("\n--- FINAL DIAGNOSTIC REPORT ---")
                print(f"Student ID: {teacher_agent.student_id}")
                
                print(json.dumps(teacher_agent.get_diagnostic_report(), indent=4)) 
                break 


            # 5. Evaluate the answer
            print(f"\nEvaluating answer for topic: {teacher_agent.get_current_topic()}...")
            
            is_correct, feedback = teacher_agent.evaluate_answer(user_input, previous_lesson, session_language)
            
            print("\n[TEACHER FEEDBACK]: " + feedback)
            print(f"Diagnostic Result: {'Correct' if is_correct else 'Incorrect'}\n")
            
            
            # --- 2. Y/N CHECK BEFORE NEXT QUESTION (नया सुधार) ---
            
            continue_action = input_func("Do you want to continue to the next question? (Y/N or Exit): ").strip().lower()
            
            if continue_action in ['n', 'exit', 'quit']:
                print("\nThank you! Your session has ended.")
                print("\n--- FINAL DIAGNOSTIC REPORT ---")
                print(f"Student ID: {teacher_agent.student_id}")
                print(json.dumps(teacher_agent.get_diagnostic_report(), indent=4))
                break # Exit here

            elif continue_action != 'y':
                print("Invalid input. Continuing to the next question.")


            # 6. Ask for the next step 
            print("\n--- Next Lesson/Question ---")
            next_lesson = teacher_agent.start_session(session_mode, session_language, exam_name) 
            previous_lesson = next_lesson
            print("\n[TEACHER]: " + next_lesson)
            
        except Exception as e:
            # Catch any error
            print(f"\nAn unexpected error occurred: {e}")
            print("\n--- ATTEMPTING FINAL DIAGNOSTIC REPORT BEFORE CRASH ---")
            print(json.dumps(teacher_agent.get_diagnostic_report(), indent=4)) 
            break

# Start the interactive session. 
interactive_chat_loop()

-----------------------------------------------------
🤖 Shiksha Saarthi (Interactive Session) 🚀
Type 'exit' or 'quit' to end the session at any time.
-----------------------------------------------------


In which language would you like to study (e.g., Hindi, English)?  english
Which mode would you like to use (Class/Competitive)?  class
Which subject would you like to study?  computer
Which topic in Computer would you like to study (Press Enter for default: Basic Concepts)?  Ai Ml



Starting Class session for: Computer: Ai Ml in english (Exam: None)...


[DIAGNOSTICS]: Suggested next topic is: Computer: Ai Ml

[Research Agent]: Attempting to search for 'Simple real-world examples for Computer: Ai Ml'...
[Research Agent]: Summary generated successfully.

[TEACHER]: Hello, my wonderful students! I'm Shiksha Saarthi, and I'm so excited to learn with you today!

Today, we're going to talk about something really amazing that computers can do. Imagine if your computer could almost **think** and **learn** new things, just like you do! Isn't that super cool? Well, that's what we call **Artificial Intelligence (AI)** and **Machine Learning (ML)**. Don't worry about the big words, let's think of them as ways computers get super smart!

Think of it like this:

*   **Talking to computers:** Have you ever heard your grown-ups talk to a smart speaker like Alexa or Siri, and ask it questions or tell it to play a song? That's AI! The computer listens to your words, understands t


[STUDENT]:  c



Evaluating answer for topic: Computer: Ai Ml...

[TEACHER FEEDBACK]: TRUE
Wonderful job! You're showing great understanding of this exciting topic.
Diagnostic Result: Correct



Do you want to continue to the next question? (Y/N or Exit):  N



Thank you! Your session has ended.

--- FINAL DIAGNOSTIC REPORT ---
Student ID: live_student
{
    "progress": [
        {
            "topic": "Computer: Ai Ml",
            "question": "Session Start",
            "answer_given": "N/A",
            "correct": false
        },
        {
            "topic": "Computer: Ai Ml",
            "question": "Hello, my wonderful students! I'm Shiksha Saarthi, and I'm so excited to learn with you today!\n\nToday, we're going to talk about something really amazing that computers can do. Imagine if your computer could almost **think** and **learn** new things, just like you do! Isn't that super cool? Well, that's what we call **Artificial Intelligence (AI)** and **Machine Learning (ML)**. Don't worry about the big words, let's think of them as ways computers get super smart!\n\nThink of it like this:\n\n*   **Talking to computers:** Have you ever heard your grown-ups talk to a smart speaker like Alexa or Siri, and ask it questions or tell it to 

Key Differentiating Features

These advanced features distinguish Shiksha Saarthi:

1. Exam-Specific Coaching: Uses the exam name (e.g., NIMCET) to ensure the question difficulty and content review are accurate and real-world competitive exam standards.


2. Contextual Solutions: If the student answers incorrectly, the agent is prompted to provide a detailed, step-by-step explanatory solution (samjha ke) instead of just feedback.


3. Multilingual Interface: The entire session can be conducted in the language of the student's choice (e.g., Hindi, English, etc.).

## Memory Demonstration: Final Diagnostic Report

The JSON output below proves that the Diagnostic Agent successfully tracked the entire Q&A session, demonstrating the Sessions & Memory feature. It logs the full history, including questions and answers, and updates the score.

```json
{
    "progress": [
        {
            "topic": "Math: lcm",
            "question": null,
            "answer_given": null,
            "correct": false
        },
        {
            "topic": "Math: probability",
            "question": null,
            "answer_given": null,
            "correct": false
        },
        {
            "topic": "Math: probability",
            "question": "Hello everyone! I am Shiksha Saarthi... [Question about apples and bananas]",
            "answer_given": "impossible",
            "correct": true
        },
        {
            "topic": "Math: probability",
            "question": "Hello, my little mathematicians! I'm your teacher... [Question about mouse baking a cake]",
            "answer_given": "A",
            "correct": false
        }
    ],
    "current_topic": "Math: probability",
    "score_total": 1,
    "question_count": 4
}